# Understanding SB 50 in LA, Part 2
Let's pick up where we left off last time. We should have prepared all the data regarding the transportation-based criteria. Now we are going to move on to the two newer criteria for SB50: Job-Rich areas (an addition) and Sensitive Communities (a subtraction). After the first exercise, you should have the following files in your `data/processed/` directory:
```
data/                             
├── processed/  
│   ├── rail_stop_buff_wgs84.geojson   <- 1/2 mi buff around rail stations
│   └── hqbus_stop_buff_wgs84.geojson  <- 1/4 mi buff around HQ bus stops            
└── raw/         
```

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
from ipyleaflet import Map, GeoData, basemaps, LayersControl

### Adding the Job-Rich Areas

A second part of the bill allowed for additional development "job rich" areas. What the legislation meant by "job rich" was not exactly clear; however, it was understood by insiders that Weiner's staff were considering a specific map within the _Mapping Opportunities in California_ project, specifically the view that contained the definition of "high-opportunity + jobs-rich, long in-commutes, and/or jobs-housing mismatch." You can find an interactive version of the map [here](http://mappingopportunityca.org/).  

However, if you take a look at the interactive map, there is not a readily-available link to download the underlying data. Might there be a way to find out where the data driving the map lives? Go ahead and examine the underlying code by right-clicking the map and selecting "Inspect". Dig through the HTML and look for the scripts labeled "js/getData.js" and "js/map.js", which _sound_ like they may have some clues to our data _(Hint: keep an eye out for 2 files: finalData.csv and a companion json file that will contain the spatial data)_.

Once you find where the data files are located, go ahead and add it to your project within the `data/raw` directory so we can use it for our project. However, before loading it, take a peek at the contents. You might notice that the format doesn't look quite the same as the GeoJSON that we've been using (there was also a hint in the .js code as well). Instead, these data are spatial, but they are TopoJSON data, which you can read more about [here](https://bost.ocks.org/mike/topology/).

Depending on your version of `fiona`, you might already have the driver installed and be able to read it directly into a GeoPandas dataframe using the following command: `jobs_shapes = gpd.read_file('data/raw/nodata.json', driver='TopoJSON')`. If not, make sure to update `fiona` to the latest version (1.8.5 or greater).

In [2]:
# Load data & set CRS (if not already set) to 4326
jobs_shapes = gpd.read_file('data/raw/nodata.json', driver='TopoJSON')
jobs_shapes.crs = {'init':'epsg:4326'}

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Now that we've loaded our spatial layers, let's take a look at what we have. Using the example from Part 1 of this exercise, go ahead and display the contents of `geos` in an `ipyleaflet` map. You should see a map that somewhat resembles this:

![TopoMap](img/topo_mapping_opportunities.png)

In [3]:
# TODO: Create basemap, zoomed out a bit to CA
m = Map(center=(34, -118), zoom = 7, basemap= basemaps.Esri.WorldTopoMap)

# TODO: Create the GeoData Object and add to the Map
jobs_gd = GeoData(geo_dataframe = jobs_shapes,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.1},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Jobs')

# Add the GeoData Object as Map Layer
m.add_layer(jobs_gd)

# Optional: Add layer control
m.add_control(LayersControl())

# Display the map
m

Map(center=[34, -118], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Now that we have the geographic boundaries, we need to perform additional manipulations:

1. **Filter for Los Angeles County**: Currently the data is for all of California. For our project, we only need LA County.
2. **Join with data**: We currently are mapping only the geography, not the data. We will need to join to `finalData.csv` 
3. **Reconstruct the appropriate view**: We are intersted in the specific view called "high-opportunity + jobs-rich, long in-commutes, and/or jobs-housing mismatch" that was identified in the _Mapping Opportunities in California_ map. We will need to figure out how to reconstruct that subset of the data (since there are other views in the map as well, all of which are derived from the same `finalData.csv`.

##### Step 1: Filter for LA County
Let's start by filtering those areas only in LA County. To apply this first filter, we are going to need a geographic boundary of LA County, which you can find [here](http://geohub.lacity.org/datasets/10f1e37c065347e693cf4e8ee753c09b_15). Write a command to query the API and save the save the result to our `data/raw` folder.

In [4]:
# TODO: Import requests & json packages
import requests, json
# TODO: Call the API to get the GeoJSON data, and save to 'data/raw'
url = 'https://opendata.arcgis.com/datasets/10f1e37c065347e693cf4e8ee753c09b_15.geojson'
resp = requests.get(url)

# Only move forward if there is a successful status code
if resp.status_code == requests.codes.ok:

    # Write out JSON to data/ or data/raw 
    with open('data/raw/lacounty_wgs84.geojson', 'w') as outfile:
        json.dump(resp.json(), outfile)

Now that we've saved our data, load it back into our current workspace. 
  
The [GeoPandas Documentation](http://geopandas.org/mergingdata.html) discusses two types of joins: attribute joins and spatial joins. Once we have our LA County boundary, we want to perform a GeoPandas _spatial join_ operation on the data, keeping all the geographies from `jobs_shapes` that are within the LA County boundary. Review the GeoPandas documentation on spatial joins [here](http://geopandas.org/mergingdata.html#spatial-joins). Confirm that the filter worked correctly by printing out the number of rows in the dataframe before and after the join.

In [5]:
# TODO: Load in the LA County Boundary data as GDF
lacounty_gdf = gpd.read_file('data/raw/lacounty_wgs84.geojson', driver='GeoJSON')

# TODO: Apply Spatial join to data
la_geo = gpd.sjoin(jobs_shapes, lacounty_gdf, how="inner", op="within")

# Print the lengths of the GDF before & after the join, confirming that rows have been dropped
print(f'There are {len(jobs_shapes)} rows in the pre-join GDF.')
print(f'There are {len(la_geo)} rows in the post-join GDF.')

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/geopandas/tools/sjoin.py:58: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  warn(


There are 8047 rows in the pre-join GDF.
There are 2243 rows in the post-join GDF.


Once you've confirmed that the post-join dataframe is smaller than the previous one, remove all the columns that were added from the LA County boundary file during the join. Once that is done, go ahead and save it to disk at `data/processed/la_geo_wgs84.geojson`.

In [6]:
# TODO: Only keep columns: 'id', 'fips', 'geometry'
la_geo = la_geo.loc[:,['id', 'fips', 'geometry']]

# TODO: Write filtered geometry to disk
la_geo.to_file("data/processed/la_geo_wgs84.geojson", driver='GeoJSON')

##### Step 2: Join the Geography to Data file
Let's join our geography file to our data to create one GeoDataFrame containing both. In this case, we will be performing an attribute join (we previously did an _attribute join_ on the GDF), based on a unique identifier for each geometry object.

Let's begin by loading our `finalData.csv` into a Pandas DataFrame and inspecting the head. Let's also inspect the head of our `geos` GeoDataFrame and look for an ID value that we could use to join the two.

In [7]:
# TODO: Load finalData.csv from data/raw 
jobs_data = pd.read_csv('data/raw/finalData.csv')

# Inspect the head of the DataFrame
jobs_data.head()

,fips,pct_white_2017,pct_hispanic_2017,pct_asian_2017,pct_black_2017,pct_other_two_race_2017,county_name,total_pop_2017,pop_density_2017,pct_above_200_pov_2017,...,flag_jobs,flag_jfit,flag_excluded,placenm,oppFlag,jobFlag,finalFlag,jfitOppFlag,jobsOppFlag,distOppFlag
0,6083002806,0.520165,0.393416,0.009465,0.035391,0.041564,Santa Barbara,2430.0,2970.660147,0.681781,...,0,0,0,Lompoc City,0,0,0,0,0,0
1,6083002913,0.823986,0.131559,0.036185,0.004135,0.004135,Santa Barbara,3869.0,4655.836342,0.812500,...,1,1,0,NaN,1,1,1,1,1,0
2,6083002914,0.619019,0.218842,0.128470,0.001181,0.032487,Santa Barbara,3386.0,4857.962697,0.911695,...,0,1,0,Goleta City,1,1,1,1,0,1
3,6083000400,0.722048,0.169470,0.070227,0.000000,0.038255,Santa Barbara,5411.0,6814.861461,0.725189,...,1,1,0,Santa Barbara City,1,1,1,1,1,1
4,6083001203,0.743480,0.181248,0.066689,0.002971,0.005612,Santa Barbara,3029.0,4460.972018,0.621112,...,1,1,0,Santa Barbara City,1,1,1,1,1,1


In [8]:
# TODO: Inspect the head of our la_geo GDF
la_geo.head()

,id,fips,geometry
1168,None,06037101110,"POLYGON ((-118.29944 34.25597, -118.30228 34.2..."
1169,None,06037101122,"POLYGON ((-118.27745 34.25992, -118.27819 34.2..."
1170,None,06037101210,"POLYGON ((-118.29944 34.25597, -118.29106 34.2..."
1171,None,06037101220,"POLYGON ((-118.27608 34.24648, -118.27944 34.2..."
1172,None,06037101300,"POLYGON ((-118.26526 34.25236, -118.26532 34.2..."


You should be able to see a field that we can use for joining. However, those especially astute will notice a slight difference betweeen the two fields; one has a leading `0` while the other does not. We can fix this by using Python's [Zfill](https://python-reference.readthedocs.io/en/latest/docs/str/zfill.html) string method (in this case, let's keep both columns as strings, wthough we also could have converted both to numeric types). Go ahead and replace the problematic column with the extra preceeding `0`, preserving the column name. Then join both dataframes to create one unified GeoDataFrame.


*Hint: You can define a simple function that uses zfill to add a 0 to each string, then use pandas' apply method to apply it to the entire column.*

*Unclear if should be using larger jobs_shapes or the filtered la_geo here.*
*Instructions in below cell imply jobs_shapes but la_geo seems to make more sense for rest of excercise.*

In [9]:
# TODO: Zfill correct column (convert to str type if needed) & replace with original
jobs_data.fips = jobs_data.fips.astype('str').apply(lambda x: x.zfill(11))

# TODO: Join data to GeoDataFrame
# la_jobs_gdf = jobs_data.set_index('fips').join(la_geo.set_index('fips'), how='inner')
la_jobs_gdf = la_geo.merge(jobs_data, on='fips')

# Print a count of the length of (1) jobs_data, (2) jobs_shapes, and (3) jobs_gdf to confirm no dropped rows

# print(f'Length of Shapes: {len(jobs_shapes)}')
## Shouldn't this be LA?

print(f'Length of Shapes: {len(la_geo)}')
print(f'Length of Data: {len(jobs_data)}')
print(f'Length of Merged DF: {len(la_jobs_gdf)}')

# TODO: Inspect the head of the new merged GDF
la_jobs_gdf.head()

Length of Shapes: 2243
Length of Data: 8057
Length of Merged DF: 2243


,id,fips,geometry,pct_white_2017,pct_hispanic_2017,pct_asian_2017,pct_black_2017,pct_other_two_race_2017,county_name,total_pop_2017,...,flag_jobs,flag_jfit,flag_excluded,placenm,oppFlag,jobFlag,finalFlag,jfitOppFlag,jobsOppFlag,distOppFlag
0,None,06037101110,"POLYGON ((-118.29944 34.25597, -118.30228 34.2...",0.541174,0.371222,0.061761,0.011389,0.014455,Los Angeles,4566.0,...,0,0,0,Los Angeles City,0,0,0,0,0,0
1,None,06037101122,"POLYGON ((-118.27745 34.25992, -118.27819 34.2...",0.815274,0.089426,0.082572,0.000000,0.012728,Los Angeles,3064.0,...,0,0,0,Los Angeles City,0,0,0,0,0,0
2,None,06037101210,"POLYGON ((-118.29944 34.25597, -118.29106 34.2...",0.411716,0.527387,0.024657,0.028463,0.007778,Los Angeles,6043.0,...,0,0,0,Los Angeles City,0,0,0,0,0,0
3,None,06037101220,"POLYGON ((-118.27608 34.24648, -118.27944 34.2...",0.504790,0.305090,0.118263,0.015868,0.055988,Los Angeles,3340.0,...,0,0,0,Los Angeles City,0,0,0,0,0,0
4,None,06037101300,"POLYGON ((-118.26526 34.25236, -118.26532 34.2...",0.817036,0.092882,0.057176,0.026138,0.006768,Los Angeles,4285.0,...,0,0,0,Los Angeles City,1,0,0,0,0,0


##### Step 3: Reconstruct the Scenario
As you can see from the [web map](https://mappingopportunityca.org/), The _Mapping Opportunities in California_ map has several different Scenarios:
* High-Opportunity
* High-Opportunity + Jobs-Rich
* High-Opportunity + Jobs-Housing Mismatch
* High-Opportunity + Long In-Commutes
* High-Opportunity + Jobs-Rich, Long In-Commutes, and/or Jobs-Housing Mismatch

We are interested only in the last scenario, which we believe would have been the basis for the "Jobs Rich" definition for SB50. Since our new merged GeoDataFrame contains all the data to construct any of those scenarios, we are going to want to apply a *filter* to get only those areas matching all conditions. Take a look at all the flag fields at the end of `jobs_gdf` and play around turning them on/off and then testing the output of the map until you match the [web map](https://mappingopportunityca.org/).

In [10]:
# TODO: Apply filters to data
la_jobs_gdf['sum_flags'] = (la_jobs_gdf['flag_above200'] +
                                    la_jobs_gdf['flag_bach'] +
                                    la_jobs_gdf['flag_emp'] +
                                    la_jobs_gdf['flag_read'] +
                                    la_jobs_gdf['flag_frpm'] +
                                    la_jobs_gdf['flag_grad'])
#Replicates their "at least four" methodology
la_jobs_filtered_gdf = la_jobs_gdf[la_jobs_gdf['sum_flags'] >= 4]

# TODO: Create a new map object and add your filtered GDF (or reuse the one before)
#       and keep testing until you get the right set of filters on your data.
m = Map(center=(34, -118), zoom = 10, basemap= basemaps.Esri.WorldTopoMap)

final_gd = GeoData(geo_dataframe = la_jobs_filtered_gdf,
                   style={'color': 'black', 'fillColor': 'green', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Jobs')

# Add the GeoData Object as Map Layer
m.add_layer(final_gd)

m

Map(center=[34, -118], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Great. Now we've filtered to areas in Los Angeles County defined as "Jobs Rich". Now that we've applied all our filters and confirmed that everything looks good, go ahead and save our final data as a geojson file `lacounty_opp_wgs84.geojson` within the `data/processed` folder.

In [11]:
# TODO: Save to data/processed
la_jobs_filtered_gdf.to_file("data/processed/lacounty_opp_wgs84.geojson", driver='GeoJSON')

## Exempt Areas
### Sensitive Communities
During the first proprosal of SB375, California Senator Scott Weiner received quite a bit of blowback from those who were worried that his proposal would lead to rapid gentrification of neighborhoods and the pushout of renters who could no longer afford their rents. To address these concerns, SB 50 included a provision that exempted certain areas from SB50 that may face adverse impacts by the bill. Sensitive Communities were defined by in the bill as:
* 'High Segregation & Poverty' or 'Low Resource' in the [TCAC Opportunity Maps](https://www.treasurer.ca.gov/ctcac/opportunity.asp)
* Areas with [CalEnviroScreen](https://oehha.ca.gov/calenviroscreen/report/calenviroscreen-30) scores in the top 25th percentile statewide

##### Step 1: Load & Filter TCAC Data
The TCAC data is provided as an excel data file that needs to be joined to census tract boundaries. Fortunately, we've already done the work of getting the census boundary file for LA County, saved as `data/processed/la_geo_wgs84.geojson`. Download the _2019 Statewide Summary Table_ from the TCAC Webpage and save the "LosAngeles" sheet as a CSV file to `data/raw/la_tcac.csv`. Then, load it back into our notebook and join it to our data. 

In [12]:
la_tcac = pd.read_csv('data/raw/la_tcac.csv')


In [13]:
# TODO: Load the LA TCAC data (after downloading and converting to CSV)
la_tcac = pd.read_csv('data/raw/la_tcac.csv')

# TODO: Perform attribute join of TCAC data to Census Tract boundaries
la_tcac = la_tcac.rename(columns={'Tract FIPS Code':'fips'})
la_tcac['fips'] = la_tcac['fips'].astype('str').apply(lambda x: x.zfill(11))

la_tcac_gdf = la_geo.merge(la_tcac, on='fips')


# Print rowcounts before & after merge
print(f'There are {len(la_geo)} rows in the pre-join LA County Census Tract File.')
print(f'There are {len(la_tcac_gdf)} rows in the post-join LA TCAC GDF.')

# TODO: Check the head of the merged GDF for final confirmation
la_tcac_gdf.head()

There are 2243 rows in the pre-join LA County Census Tract File.
There are 2224 rows in the post-join LA TCAC GDF.


,id,fips,geometry,County,County ID,Economic Domain Average Z Scores (by region),Environmental Domain Average Z Scores (by region),Education Domain Average Z Scores (by region),Composite Index Score,Final Category,Unnamed: 8
0,None,06037101110,"POLYGON ((-118.29944 34.25597, -118.30228 34.2...",Los Angeles,6037.0,-0.460,0.356,-0.250,-0.118,Moderate Resource,NaN
1,None,06037101122,"POLYGON ((-118.27745 34.25992, -118.27819 34.2...",Los Angeles,6037.0,0.186,0.413,-0.250,0.116,High Resource,NaN
2,None,06037101210,"POLYGON ((-118.29944 34.25597, -118.29106 34.2...",Los Angeles,6037.0,-0.634,0.334,-0.250,-0.183,Moderate Resource,NaN
3,None,06037101220,"POLYGON ((-118.27608 34.24648, -118.27944 34.2...",Los Angeles,6037.0,-0.552,0.312,-0.266,-0.169,Moderate Resource,NaN
4,None,06037101300,"POLYGON ((-118.26526 34.25236, -118.26532 34.2...",Los Angeles,6037.0,-0.179,0.066,0.574,0.154,High Resource,NaN


Sensitive communities are defined as those that have the either the designation of 'High Segregation and Poverty' or 'Low Resource'. Let's filter for those two labels in the 'Final Category' column of our GeoDataFrame and then save to disk as `data/processed/la_tcac_filtered_wgs84.geojson`.

In [14]:
la_tcac_gdf['Final Category'].unique()

array(['Moderate Resource', 'High Resource', 'Low Resource',
       'High Segregation & Poverty', 'Highest Resource', nan],
      dtype=object)

In [15]:
# TODO: Filter GDF
la_tcac_filtered_gdf = la_tcac_gdf[la_tcac_gdf['Final Category'].isin(
    ['High Segregation & Poverty', 'Low Resource'])]

# TODO: Print rowcounts before/after filter for confirmation
print(f'There are {len(la_tcac_gdf)} rows in the pre-filter LA TCAC GDF')
print(f'There are {len(la_tcac_filtered_gdf)} rows in the post-filter LA TCAC GDF.')
# TODO: Save to disk
la_tcac_filtered_gdf.to_file('data/processed/la_tcac_filtered_wgs84.geojson', driver='GeoJSON')

There are 2224 rows in the pre-filter LA TCAC GDF
There are 848 rows in the post-filter LA TCAC GDF.


##### Step 2: Load CalEnviroScreen Data
All of the data used in the calculation of the CalEnviroScreen scores can be found [here](https://oehha.ca.gov/calenviroscreen/report/calenviroscreen-30). There are a few different formats provided: Shapefile, ArcGIS GeoDatabase, Google Earth KML file, and Spreadsheet. Since we already have census tracts, we can join the values from the spreadsheet to our geojson census tracts. Download the `ces3results.xlsx` file, save the first sheet ('CES 3.0 2018 Update') as `data/raw/ces3results_2018update.csv`, then load it into our notebook and join with our LA Census Tract GeoJSON file.

In [16]:
ces3results = pd.read_csv('data/raw/ces3results_2018update.csv')

In [17]:
# TODO: Load data into notebook
ces3results = pd.read_csv('data/raw/ces3results_2018update.csv')

# TODO: Join to la tracts
ces3results = ces3results.rename(columns={'Census Tract':'fips'})
ces3results['fips'] = ces3results['fips'].astype('str').apply(lambda x: x.zfill(11))

ces3_gdf = la_geo.merge(ces3results, on='fips')

# TODO: Confirm the join by printing the rowcount and examining the head of the mereged GDF
print(f'There are {len(la_geo)} rows in the pre-join LA County Census Tract File.')
print(f'There are {len(ces3_gdf)} rows in the post-join LA CES3 GDF.')

# TODO: Check the head of the merged GDF for final confirmation
ces3_gdf.head()

There are 2243 rows in the pre-join LA County Census Tract File.
There are 2243 rows in the post-join LA CES3 GDF.


,id,fips,geometry,Total Population,California County,ZIP,Nearby City \n(to help approximate location only),Longitude,Latitude,CES 3.0 Score,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,None,06037101110,"POLYGON ((-118.29944 34.25597, -118.30228 34.2...",4731,Los Angeles,91042,Tujunga,-118.292987,34.259474,22.67,...,64.63,41.6,61.94,8.8,45.97,29.7,87.19,56.12,5.82,59.24
1,None,06037101122,"POLYGON ((-118.27745 34.25992, -118.27819 34.2...",3664,Los Angeles,91042,Tujunga,-118.290147,34.267721,11.88,...,21.80,7.5,3.43,8.2,40.51,10.1,12.86,34.77,3.61,26.61
2,None,06037101210,"POLYGON ((-118.29944 34.25597, -118.29106 34.2...",5990,Los Angeles,91042,Tujunga,-118.290731,34.252972,28.88,...,87.23,55.6,79.75,9.5,51.87,26.0,79.40,66.16,6.86,74.39
3,None,06037101220,"POLYGON ((-118.27608 34.24648, -118.27944 34.2...",3363,Los Angeles,91042,Tujunga,-118.281632,34.251609,27.12,...,87.47,37.6,56.23,8.0,38.51,32.2,91.36,59.99,6.22,64.94
4,None,06037101300,"POLYGON ((-118.26526 34.25236, -118.26532 34.2...",4199,Los Angeles,91042,Tujunga,-118.270999,34.248778,19.06,...,35.77,19.4,25.53,8.8,45.97,12.0,21.22,33.49,3.47,24.50


Now that we have our GeoDataFrame with CalEnviroScreen data, let's go ahead and filter for the top CES 3.0 25 Percentile range, which is the critera for excluding areas from the impacts of SB50. This is especially easy since you'll notice there is already a Yes/No Field in the 'SB 535 Disadvantaged Community' Column. Let's filter for those areas that fall within this category.

In [18]:
ces3_gdf.columns

Index(['id', 'fips', 'geometry', 'Total Population', 'California County',
       'ZIP', 'Nearby City \n(to help approximate location only)', 'Longitude',
       'Latitude', 'CES 3.0 Score', ' CES 3.0 Percentile',
       'CES 3.0 \nPercentile Range', 'SB 535 Disadvantaged Community', 'Ozone',
       'Ozone Pctl', 'PM2.5', 'PM2.5 Pctl', 'Diesel PM', 'Diesel PM Pctl',
       'Drinking Water', 'Drinking Water Pctl', 'Pesticides',
       'Pesticides Pctl', 'Tox. Release', 'Tox. Release Pctl', 'Traffic',
       'Traffic Pctl', 'Cleanup Sites', 'Cleanup Sites Pctl',
       'Groundwater Threats', 'Groundwater Threats Pctl', 'Haz. Waste',
       'Haz. Waste Pctl', 'Imp. Water Bodies', 'Imp. Water Bodies Pctl',
       'Solid Waste', 'Solid Waste Pctl', 'Pollution Burden',
       'Pollution Burden Score', 'Pollution Burden Pctl', 'Asthma',
       'Asthma Pctl', 'Low Birth Weight', 'Low Birth Weight Pctl',
       'Cardiovascular Disease', 'Cardiovascular Disease Pctl', 'Education',
       'Educati

In [19]:
# TODO: Filter for disadvantaged communities
ces3_filtered_gdf = ces3_gdf[ces3_gdf['SB 535 Disadvantaged Community'] == 'Yes']

# Print the rowcount before and after to confirm the filter
print(f'There are {len(ces3_gdf)} in the pre-filtered ces3 data.')
print(f'There are {len(ces3_filtered_gdf)} in the post-filtered ces3 data.')

There are 2243 in the pre-filtered ces3 data.
There are 1026 in the post-filtered ces3 data.


Finally, let's write out our filtered dataset to `data/processed/la_ces3_filtered_wgs84.geojson`.

In [20]:
# TODO: Save to disk
ces3_filtered_gdf.to_file('data/processed/la_ces3_filtered_wgs84.geojson', driver='GeoJSON')

### Very High Fire Hazard Severity Zones
SB50 also excludes those areas deemed by CALFIRE as being within a Very High Fire Hazard Severity Zone. CALFIRE includes maps and GIS information regarding these zones on [their website](https://osfm.fire.ca.gov/divisions/wildfire-prevention-planning-engineering/wildland-hazards-building-codes/fire-hazard-severity-zones-maps/). Scroll down to find the data specific to LA County, and download the GIS (Shapefile) files for both the State Responsibility Area and Local Responsibility Area. Save them both to your `data/raw` folder and then read them back in as GDF objects. _Hint: Make sure you select the right driver!_ 

In [21]:
# TODO: Read in both shapefiles
local_firehazard_gdf = gpd.read_file('data/raw/c19fhszl06_5/c19fhszl06_5.shp', driver='ESRI Shapefile')
state_firehazard_gdf = gpd.read_file('data/raw/fhszs19sn/fhszs06_3_19.shp', driver='ESRI Shapefile')

# TODO: Examine the head of one of the GDFs
local_firehazard_gdf.head()

,OBJECTID,FID_c19fhs,HAZ_CODE,HAZ_CLASS,SRA,INCORP,VH_REC,Shape_Leng,Shape_Area,geometry
0,1,0,3,Very High,LRA,None,None,5415.460854,1.355567e+06,"POLYGON ((152996.831 -469302.197, 152996.863 -..."
1,2,1,3,Very High,LRA,None,None,2802.070818,4.236575e+05,"POLYGON ((153701.703 -468506.003, 153703.873 -..."
2,3,2,3,Very High,LRA,None,None,802.128633,3.227196e+04,"POLYGON ((191491.019 -449977.144, 191494.316 -..."
3,4,3,3,Very High,LRA,None,None,1096.587821,4.080046e+04,"POLYGON ((182453.778 -445649.110, 182216.326 -..."
4,5,4,3,Very High,LRA,None,None,59225.977459,9.379764e+06,"MULTIPOLYGON (((198200.954 -446611.203, 198199..."


Check the CRS of each of the files and re-project if needed. 

In [22]:
# Check CRS and make necessary conversions
print(local_firehazard_gdf.crs)
print(state_firehazard_gdf.crs)

# TODO: Reproject to 4326
local_firehazard_gdf = local_firehazard_gdf.to_crs('epsg:4326')
state_firehazard_gdf = state_firehazard_gdf.to_crs('epsg:4326')

epsg:3310
PROJCS["NAD_1983_Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


We want to filter both for "Very High Fire Hazard". Start by printing the unique values for "HAZ_CLASS", and then filter by the appropriate one. Also, since for our purposes we do not care about responsibility, let's union both of the geometries into one GeoDataFrame and write it out to `data/processed/highfirehazard_wgs84.geojson`.

In [23]:
# Print unique values for HAZ_CLASS for each
print(state_firehazard_gdf.HAZ_CLASS.unique())
print(local_firehazard_gdf.HAZ_CLASS.unique())

['Moderate' 'High' 'Very High']
['Very High']


*Two cells easier here*

In [24]:
# TODO: Filter for the appropriate HAZ Class
state_highfirehazard_gdf = state_firehazard_gdf[state_firehazard_gdf['HAZ_CLASS'] == 'Very High']

# Print Record Counts
print(f'There were {len(state_firehazard_gdf)} records in the pre-filtered state firehazard gdf.')
print(f'There are {len(state_highfirehazard_gdf)} records in the post-filtered state firehazard gdf.')
print(f'There are {len(local_firehazard_gdf)} records in the local firehazard gdf.')

# TODO: Concatenate both GDFs
gdf_list = [local_firehazard_gdf, state_firehazard_gdf]
lacounty_highfirehazard_gdf = pd.concat(gdf_list)

# Confirm that record count total = record count gdf1 + record count gdf2
print(f'The record count of the concat of both GDFs is {len(lacounty_highfirehazard_gdf)}.')

# TODO: Write out to data/processed


There were 1234 records in the pre-filtered state firehazard gdf.
There are 981 records in the post-filtered state firehazard gdf.
There are 161 records in the local firehazard gdf.
The record count of the concat of both GDFs is 1395.


In [25]:
type(lacounty_highfirehazard_gdf)

geopandas.geodataframe.GeoDataFrame

In [26]:
lacounty_highfirehazard_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Let's go ahead and make a map to view the geometries we just created using the `ipyleaflet` library as we did earlier with the jobs-rich areas.

In [27]:
# TODO: Create a new map object and add your filtered GDF (or reuse the one before)
#       and keep testing until you get the right set of filters on your data.
m3 = Map(center=(34, -118), zoom = 10, basemap= basemaps.Esri.WorldTopoMap)

hazard_gd = GeoData(geo_dataframe = lacounty_highfirehazard_gdf,
                   style={'color': 'black', 'fillColor': 'red', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Jobs')

# Add the GeoData Object as Map Layer
m3.add_layer(hazard_gd)

m3

Map(center=[34, -118], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### To Be Continued...
We've now processed the transportation and non-tranpsortation criteria for consideration of SB50. In the next part, we will begin putting these pieces together. Stay tuned.